<h3>General Imports</h3>

In [13]:
import os
import sys
import time
sys.path.append('/home/jbq/repositories/sans-backend/sans-backend2')
# Mantid imports
from mantid.simpleapi import LoadEventNexus, Rebin, CreateWorkspace

# drtsans imports
from drtsans.pixel_calibration import calculate_apparent_tube_width, load_calibration, as_intensities
from drtsans.tubecollection import TubeCollection

In [3]:
# "plot_detector" is a utility plotting function that we will use a couple of times
%matplotlib notebook
from drtsans.plots import plot_detector
def plot_detector(input_workspace, axes_mode='tube-pixel', panel_name='detector1'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})

<h3>Dataset</h3>

The flood file is an event nexus file 1.6GB in size. We don't need the events, but only the total intensity per pixel, so we convert to a matrix workspace and keep only one bin.

In [4]:
LoadEventNexus(Filename='/HFIR/CG2/IPTS-23801/nexus/CG2_8143.nxs.h5', OutputWorkspace='flood_workspace')
Rebin(InputWorkspace='flood_workspace', OutputWorkspace='flood_workspace',
      Params=[0, 1.E06, 1.E06], PreserveEvents=False)

Workspace2D
Title: Flood_center
Histograms: 49152
Bins: 1
Histogram
X axis: Time-of-flight / microsecond
Y axis: Counts
Distribution: False
Instrument: CG2 (2019-Oct-01 to 2100-Dec-31)Instrument from: /HFIR/CG2/IPTS-23801/nexus/CG2_8143.nxs.h5

Parameters from: /home/jbq/repositories/mantidproject/mantid0/instrument/CG2_Parameters.xml
Run start: 2020-Jan-30 23:03:03
Run end:  2020-Jan-31 01:03:03

<h3>Calibration</h3>
We carry out the calibration and apply it to the flood file to adjust for tube width, saving to a different file for comparison of workspaces before and after the calibration.

In [8]:
start_time = time.time()
calibration = calculate_apparent_tube_width('flood_workspace', load_barscan_calibration=False)
print('Calibration took ', int(time.time() - start_time), 'seconds')
calibration.apply('flood_workspace', output_workspace='calibrated_workspace')

Calibration took  43 seconds


Workspace2D
Title: Flood_center
Histograms: 49152
Bins: 1
Histogram
X axis: Time-of-flight / microsecond
Y axis: Counts
Distribution: False
Instrument: CG2 (2019-Oct-01 to 2100-Dec-31)Instrument from: /HFIR/CG2/IPTS-23801/nexus/CG2_8143.nxs.h5

Parameters from: /home/jbq/repositories/mantidproject/mantid0/instrument/CG2_Parameters.xml
Run start: 2020-Jan-30 23:03:03
Run end:  2020-Jan-31 01:03:03

<h3>Intensities Normalized by Pixel Width</h3>
In function $linear_density$ we integrate the total intensity per tube and divide by the tube width. Front end tubes collect more intentity than the back tubes. Similarly, front end tubes have a larger apparent tube width than back tubes. The ratio of total intensity to width should be similar for front and end tubes after the calibration.

In [11]:
def linear_density(workspace):
    r"""Tube total intensity per unit length of tube width"""
    collection = TubeCollection(workspace, 'detector1').sorted(view='decreasing X')
    intensities = np.array([np.sum(tube.readY) for tube in collection])
    widths = np.array([tube[0].width for tube in collection])
    return list(intensities / widths)
uncalibrated_densities = linear_density('flood_workspace')
calibrated_densities = linear_density('calibrated_workspace')

We store both linear densities in a workspace, and then we'll use matplotlib to plot both densities

In [14]:
number_tubes = len(uncalibrated_densities)
CreateWorkspace(DataX=range(number_tubes),
                DataY=np.array([uncalibrated_densities, calibrated_densities]),
                NSpec=2,   # two histograms
                Outputworkspace='linear_densities')

Workspace2D
Title: 
Histograms: 2
Bins: 192
Data points
X axis:  / 
Y axis: 
Distribution: False
Instrument: None
Run start: not available
Run end:  not available

<h3>Caveats</h3>
As it turns out, the bar is not covering the last tube. We verify this by plotting the intensities on the wing detector for the middle scan.

We have created a **mask file** whereby we mask the last tube. This flags this tube as faulty when performing the barscan calculations. **Average** pixel positions and heights will be used to estimate the calibration of this faulty tube.

In [ ]:
middle_scan_index = int((last_run - first_run) / 2)
LoadEventNexus(barscan_files[middle_scan_index], OutputWorkspace='middle_scan')
plot_wing_detector('middle_scan')
mask_file = '/home/jbq/repositories/sans-backend/sans-backend2/notebooks/barscan/biosans_mask_bank88_tube4.xml'

In [ ]:
# Carry out the calibration
start_time = time.time()
calibration = calculate_barscan_calibration(barscan_files[::3], component=detector_array,
                                            formula=formula, mask=mask_file)
print('Calibration took ', int((time.time() - start_time) / 60), 'minutes')

<h3>Saving the Calibration</h3>
There are default database files for each instrument when saving a calibration. For BIOSANS is <code>/HFIR/CG3/shared/calibration/pixel_calibration.json</code>. We don't want to mess with the official database so we save this calibration to a temporary database file.

In [ ]:
calibration.save(database='/HFIR/CG3/shared/tmp/calibration.json')

<h3>Applying a Saved Calibration</h3>
Next we use one of the barscan files as the target for our recently saved calibration.
Notice that we are applying the calibration to our input workspace and saving the result to an output workspace. If you want to <b>overwrite</b> the input workspace, then omit the <code>output_workspace</code> argument.

In [ ]:
LoadEventNexus(os.path.join(data_dir, f'CG3_895.nxs.h5'), OutputWorkspace='input_workspace')
saved_calibration = load_calibration('input_workspace', 'BARSCAN', component='wing_detector',
                                     database='/HFIR/CG3/shared/tmp/calibration.json')
start_time = time.time()
saved_calibration.apply('input_workspace', output_workspace='output_workspace')
print('Applying the calibration took ', time.time() - start_time, 'seconds')

<h3>Visualizing the Effects of the Calibration</h3>
We plot intensities on the wing detector before and after the calibration. Notice we use argument <code>axes_mode='xy'</code> that instruct <code>plot_wing_detector</code> to plot intensities versus $X$ and $Y$ coordinates, instead of the default plotting. The default plotting is versus tube index and pixel index.

Notice: generating the plots versus $X$ and $Y$ will take about a minute

In [ ]:
plot_wing_detector('input_workspace', axes_mode='xy')  # before calibration
plot_wing_detector('output_workspace', axes_mode='xy')  # after calibration

If the calibration has worked, the bar should appear more **levelled** after the calibration.

Also notice that all values of $X$ are negative, as the wing detector is standing on the negative side of the X-axis

<h3>Viewing the Calibrated Positions and Heights</h3>
We can retrieve the pixel positions and heights from the calibrated <code>output_workspace</code>. The cell below will take for each pixel its vertical position and store this value as the intensity value in workspace <code>views.positions</code>. Later we can color plot this workspace to view the assigned positions to each pixel. The same process is done for pixel heights.

Again, extracting the ~50K positions and heights takes time.

In [ ]:
views = as_intensities('output_workspace', component='wing_detector')
plot_wing_detector(views.positions)
plot_wing_detector(views.heights)